In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math

from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.pipeline import make_union

/Users/sameriliya/.pyenv/versions/lewagon/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df_X = pd.read_csv('../raw_data/initial_X_train.csv')

In [3]:
df_X.head()

,Unnamed: 0,FlightDate,Year,Quarter,Month,DayofMonth,DayOfWeek,Airline,Operating_Airline,Marketing_Airline_Network,...,Dest,CRSDepTime,OriginAirportID,OriginCityName,OriginStateName,DestAirportID,DestCityName,DestStateName,CRSArrTime,Distance
0,4324343,2018-07-20,2018,3,7,20,5,Southwest Airlines Co.,WN,WN,...,DAL,1200,12889,"Las Vegas, NV",Nevada,11259,"Dallas, TX",Texas,1640,1067.0
1,28169346,2022-01-02,2022,1,1,2,7,Spirit Air Lines,NK,NK,...,MSY,2110,15304,"Tampa, FL",Florida,13495,"New Orleans, LA",Louisiana,2148,488.0
2,18078508,2020-02-11,2020,1,2,11,2,Air Wisconsin Airlines Corp,ZW,UA,...,ORF,930,13930,"Chicago, IL",Illinois,13931,"Norfolk, VA",Virginia,1245,717.0
3,23669089,2021-02-10,2021,1,2,10,3,United Air Lines Inc.,UA,UA,...,IAH,700,15304,"Tampa, FL",Florida,12266,"Houston, TX",Texas,828,787.0
4,14530840,2020-03-12,2020,1,3,12,4,JetBlue Airways,B6,B6,...,BOS,1324,11278,"Washington, DC",Virginia,10721,"Boston, MA",Massachusetts,1455,399.0


In [4]:
dest_df = df_X[df_X['Dest'].map(df_X['Dest'].value_counts()) < 5]

In [5]:
dest_df['Dest'].value_counts()

GFK    4
GTR    4
FLO    4
ELM    4
MGM    4
      ..
PSE    1
GCC    1
LYH    1
MVY    1
TYR    1
Name: Dest, Length: 142, dtype: int64

In [6]:
df_X.columns

Index(['Unnamed: 0', 'FlightDate', 'Year', 'Quarter', 'Month', 'DayofMonth',
       'DayOfWeek', 'Airline', 'Operating_Airline',
       'Marketing_Airline_Network', 'Flight_Number_Marketing_Airline',
       'Origin', 'Dest', 'CRSDepTime', 'OriginAirportID', 'OriginCityName',
       'OriginStateName', 'DestAirportID', 'DestCityName', 'DestStateName',
       'CRSArrTime', 'Distance'],
      dtype='object')

# Function Parts

In [7]:
def data_cleaning(X):
    try:
        X = X.drop(columns = ['Unnamed: 0','Airline','Operating_Airline', 'Flight_Number_Marketing_Airline',
              'OriginStateName', 'OriginCityName','DestStateName', 'DestCityName', 'DestAirportID', 'OriginAirportID'])
        X = X.drop_duplicates()
        
    except:pass
    
    return X
   

In [8]:
df_X = data_cleaning(df_X)

In [9]:
df_X

,FlightDate,Year,Quarter,Month,DayofMonth,DayOfWeek,Marketing_Airline_Network,Origin,Dest,CRSDepTime,CRSArrTime,Distance
0,2018-07-20,2018,3,7,20,5,WN,LAS,DAL,1200,1640,1067.0
1,2022-01-02,2022,1,1,2,7,NK,TPA,MSY,2110,2148,488.0
2,2020-02-11,2020,1,2,11,2,UA,ORD,ORF,930,1245,717.0
3,2021-02-10,2021,1,2,10,3,UA,TPA,IAH,700,828,787.0
4,2020-03-12,2020,1,3,12,4,B6,DCA,BOS,1324,1455,399.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2019-09-10,2019,3,9,10,2,DL,DTW,STL,840,918,440.0
9996,2018-03-20,2018,1,3,20,2,UA,DEN,ORD,1121,1446,888.0
9997,2019-09-03,2019,3,9,3,2,WN,DEN,BUR,2015,2145,850.0
9998,2020-01-02,2020,1,1,2,4,AA,DFW,BDL,1832,2254,1471.0


## Distance Scaling

In [12]:
dist_min = df_X['Distance'].min()
dist_max = df_X['Distance'].max()

In [13]:
distance_pipe = make_pipeline(
    FunctionTransformer(lambda dist: (dist - dist_min)/(dist_max - dist_min)))
distance_pipe

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function <lambda> at 0x14bb283a0>))])

## Time Formatting/Scaling

### Day, Month, Quarter

In [15]:
df_X

,FlightDate,Year,Quarter,Month,DayofMonth,DayOfWeek,Marketing_Airline_Network,Origin,Dest,CRSDepTime,CRSArrTime,Distance
0,2018-07-20,2018,3,7,20,5,WN,LAS,DAL,1200,1640,1067.0
1,2022-01-02,2022,1,1,2,7,NK,TPA,MSY,2110,2148,488.0
2,2020-02-11,2020,1,2,11,2,UA,ORD,ORF,930,1245,717.0
3,2021-02-10,2021,1,2,10,3,UA,TPA,IAH,700,828,787.0
4,2020-03-12,2020,1,3,12,4,B6,DCA,BOS,1324,1455,399.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2019-09-10,2019,3,9,10,2,DL,DTW,STL,840,918,440.0
9996,2018-03-20,2018,1,3,20,2,UA,DEN,ORD,1121,1446,888.0
9997,2019-09-03,2019,3,9,3,2,WN,DEN,BUR,2015,2145,850.0
9998,2020-01-02,2020,1,1,2,4,AA,DFW,BDL,1832,2254,1471.0


In [10]:
df_X['FlightDate'] = pd.to_datetime(df_X["FlightDate"])
# df_X['FlightDate'].dt.dayofweek

def get_day_of_week(X):
    df_X['FlightDate'].dt.dayofweek
    
    return (df_X['FlightDate'].dt.dayofweek) + 1

In [9]:
get_day_of_week(df_X)

0       5
1       7
2       2
3       3
4       4
       ..
9995    2
9996    2
9997    2
9998    4
9999    5
Name: FlightDate, Length: 10000, dtype: int64

In [11]:
def transform_time_features(X: pd.DataFrame):
    dow = df_X['DayOfWeek']
    sin_dow = np.sin(2 * math.pi / 7 * dow)
    cos_dow = np.cos(2 * math.pi / 7 * dow)
    
    dom = df_X['DayofMonth']
    sin_dom = np.sin(2 * math.pi / 31 * dom)
    cos_dom = np.cos(2 * math.pi / 31 * dom)
    
    month = df_X['Month']
    sin_month = np.sin(2 * math.pi / 12 * month)
    cos_month = np.cos(2 * math.pi / 12 * month)
    
    qua = df_X['Quarter']
    sin_qua = np.sin(2 * math.pi / 4 * qua)
    cos_qua = np.cos(2 * math.pi / 4 * qua)
    
    dep = df_X['CRSDepTime']
    sin_dep = np.sin(2 * math.pi / 2400 * qua)
    cos_dep = np.cos(2 * math.pi / 2400 * qua)
    
    arr = df_X['CRSArrTime']
    sin_arr = np.sin(2 * math.pi / 2400 * qua)
    cos_arr = np.cos(2 * math.pi / 2400 * qua)
    
    #return np.stack([sin_dow,cos_dow, sin_dom, cos_dom, sin_month, cos_month, sin_qua, cos_qua])
    result = pd.DataFrame([sin_dow,cos_dow,sin_dom, cos_dom, sin_month, cos_month, sin_qua, cos_qua, sin_dep, 
                      cos_dep, sin_arr, cos_arr]).T
    result.columns = ['sin_dow','cos_dow','sin_dom', 'cos_dom', 'sin_month', 'cos_month', 'sin_qua', 'cos_qua', 'sin_dep', 
                      'cos_dep', 'sin_arr', 'cos_arr']
    return result

In [12]:
df_time = transform_time_features(df_X)

In [13]:
df_time

,sin_dow,cos_dow,sin_dom,cos_dom,sin_month,cos_month,sin_qua,cos_qua,sin_dep,cos_dep,sin_arr,cos_arr
0,-9.749279e-01,-0.222521,-0.790776,-0.612106,-0.500000,-8.660254e-01,-1.0,-1.836970e-16,0.007854,0.999969,0.007854,0.999969
1,-2.449294e-16,1.000000,0.394356,0.918958,0.500000,8.660254e-01,1.0,6.123234e-17,0.002618,0.999997,0.002618,0.999997
2,9.749279e-01,-0.222521,0.790776,-0.612106,0.866025,5.000000e-01,1.0,6.123234e-17,0.002618,0.999997,0.002618,0.999997
3,4.338837e-01,-0.900969,0.897805,-0.440394,0.866025,5.000000e-01,1.0,6.123234e-17,0.002618,0.999997,0.002618,0.999997
4,-4.338837e-01,-0.900969,0.651372,-0.758758,1.000000,6.123234e-17,1.0,6.123234e-17,0.002618,0.999997,0.002618,0.999997
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9.749279e-01,-0.222521,0.897805,-0.440394,-1.000000,-1.836970e-16,-1.0,-1.836970e-16,0.007854,0.999969,0.007854,0.999969
9996,9.749279e-01,-0.222521,-0.790776,-0.612106,1.000000,6.123234e-17,1.0,6.123234e-17,0.002618,0.999997,0.002618,0.999997
9997,9.749279e-01,-0.222521,0.571268,0.820763,-1.000000,-1.836970e-16,-1.0,-1.836970e-16,0.007854,0.999969,0.007854,0.999969
9998,-4.338837e-01,-0.900969,0.394356,0.918958,0.500000,8.660254e-01,1.0,6.123234e-17,0.002618,0.999997,0.002618,0.999997


In [15]:
df_X = pd.merge(df_X, df_time, left_index=True, right_index=True, how = 'outer')

In [16]:
df_X

,FlightDate,Year,Quarter,Month,DayofMonth,DayOfWeek,Marketing_Airline_Network,Origin,Dest,CRSDepTime,...,sin_dom,cos_dom,sin_month,cos_month,sin_qua,cos_qua,sin_dep,cos_dep,sin_arr,cos_arr
0,2018-07-20,2018,3,7,20,5,WN,LAS,DAL,1200,...,-0.790776,-0.612106,-0.500000,-8.660254e-01,-1.0,-1.836970e-16,0.007854,0.999969,0.007854,0.999969
1,2022-01-02,2022,1,1,2,7,NK,TPA,MSY,2110,...,0.394356,0.918958,0.500000,8.660254e-01,1.0,6.123234e-17,0.002618,0.999997,0.002618,0.999997
2,2020-02-11,2020,1,2,11,2,UA,ORD,ORF,930,...,0.790776,-0.612106,0.866025,5.000000e-01,1.0,6.123234e-17,0.002618,0.999997,0.002618,0.999997
3,2021-02-10,2021,1,2,10,3,UA,TPA,IAH,700,...,0.897805,-0.440394,0.866025,5.000000e-01,1.0,6.123234e-17,0.002618,0.999997,0.002618,0.999997
4,2020-03-12,2020,1,3,12,4,B6,DCA,BOS,1324,...,0.651372,-0.758758,1.000000,6.123234e-17,1.0,6.123234e-17,0.002618,0.999997,0.002618,0.999997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2019-09-10,2019,3,9,10,2,DL,DTW,STL,840,...,0.897805,-0.440394,-1.000000,-1.836970e-16,-1.0,-1.836970e-16,0.007854,0.999969,0.007854,0.999969
9996,2018-03-20,2018,1,3,20,2,UA,DEN,ORD,1121,...,-0.790776,-0.612106,1.000000,6.123234e-17,1.0,6.123234e-17,0.002618,0.999997,0.002618,0.999997
9997,2019-09-03,2019,3,9,3,2,WN,DEN,BUR,2015,...,0.571268,0.820763,-1.000000,-1.836970e-16,-1.0,-1.836970e-16,0.007854,0.999969,0.007854,0.999969
9998,2020-01-02,2020,1,1,2,4,AA,DFW,BDL,1832,...,0.394356,0.918958,0.500000,8.660254e-01,1.0,6.123234e-17,0.002618,0.999997,0.002618,0.999997


## Categorical Encoding

In [21]:
df_X.head()

,FlightDate,Year,Quarter,Month,DayofMonth,DayOfWeek,Marketing_Airline_Network,Origin,Dest,CRSDepTime,...,sin_dom,cos_dom,sin_month,cos_month,sin_qua,cos_qua,sin_dep,cos_dep,sin_arr,cos_arr
0,2018-07-20,2018,3,7,20,5,WN,LAS,DAL,1200,...,-0.790776,-0.612106,-0.500000,-8.660254e-01,-1.0,-1.836970e-16,0.007854,0.999969,0.007854,0.999969
1,2022-01-02,2022,1,1,2,7,NK,TPA,MSY,2110,...,0.394356,0.918958,0.500000,8.660254e-01,1.0,6.123234e-17,0.002618,0.999997,0.002618,0.999997
2,2020-02-11,2020,1,2,11,2,UA,ORD,ORF,930,...,0.790776,-0.612106,0.866025,5.000000e-01,1.0,6.123234e-17,0.002618,0.999997,0.002618,0.999997
3,2021-02-10,2021,1,2,10,3,UA,TPA,IAH,700,...,0.897805,-0.440394,0.866025,5.000000e-01,1.0,6.123234e-17,0.002618,0.999997,0.002618,0.999997
4,2020-03-12,2020,1,3,12,4,B6,DCA,BOS,1324,...,0.651372,-0.758758,1.000000,6.123234e-17,1.0,6.123234e-17,0.002618,0.999997,0.002618,0.999997


In [18]:
#ohe1 = OneHotEncoder(sparse = False) 

#ohe1.fit(df_X[['Marketing_Airline_Network']]) 

#df_X[ohe1.get_feature_names_out()] = ohe1.transform(df_X[['Marketing_Airline_Network']])

#df_X.drop(columns = ["Marketing_Airline_Network"], inplace = False)  

##df_X.head() 

In [22]:
cat_transformer = OneHotEncoder(handle_unknown='ignore', sparse=False)

cat_pipeline = make_column_transformer(
    (cat_transformer, ['Marketing_Airline_Network', 'Origin', 'Dest', 'Year']),
    remainder='passthrough')

#pd.DataFrame(cat_pipeline.fit_transform(df_X)).head()

## Full Pipeline

In [23]:
preprocessor = ColumnTransformer(
[("dist_preproc", distance_pipe, ['Distance']),
    ],
)
preprocessor

ColumnTransformer(transformers=[('dist_preproc',
                                 Pipeline(steps=[('functiontransformer',
                                                  FunctionTransformer(func=<function <lambda> at 0x14bb283a0>))]),
                                 ['Distance'])])

In [24]:
preproc_full = make_union(preprocessor, cat_pipeline)

In [25]:
X_processed = pd.DataFrame(preproc_full.fit_transform(df_X))

In [26]:
X_processed

,0,1,2,3,4,5,6,7,8,9,...,690,691,692,693,694,695,696,697,698,699
0,0.206966,0,0,0,0,0,0,0,0,0,...,-0.790776,-0.612106,-0.5,-0.866025,-1,-1.83697e-16,0.0078539,0.999969,0.0078539,0.999969
1,0.0897124,0,0,0,0,0,0,0,1,0,...,0.394356,0.918958,0.5,0.866025,1,6.12323e-17,0.00261799,0.999997,0.00261799,0.999997
2,0.136087,0,0,0,0,0,0,0,0,1,...,0.790776,-0.612106,0.866025,0.5,1,6.12323e-17,0.00261799,0.999997,0.00261799,0.999997
3,0.150263,0,0,0,0,0,0,0,0,1,...,0.897805,-0.440394,0.866025,0.5,1,6.12323e-17,0.00261799,0.999997,0.00261799,0.999997
4,0.0716889,0,0,1,0,0,0,0,0,0,...,0.651372,-0.758758,1,6.12323e-17,1,6.12323e-17,0.00261799,0.999997,0.00261799,0.999997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.0799919,0,0,0,1,0,0,0,0,0,...,0.897805,-0.440394,-1,-1.83697e-16,-1,-1.83697e-16,0.0078539,0.999969,0.0078539,0.999969
9996,0.170717,0,0,0,0,0,0,0,0,1,...,-0.790776,-0.612106,1,6.12323e-17,1,6.12323e-17,0.00261799,0.999997,0.00261799,0.999997
9997,0.163021,0,0,0,0,0,0,0,0,0,...,0.571268,0.820763,-1,-1.83697e-16,-1,-1.83697e-16,0.0078539,0.999969,0.0078539,0.999969
9998,0.288781,1,0,0,0,0,0,0,0,0,...,0.394356,0.918958,0.5,0.866025,1,6.12323e-17,0.00261799,0.999997,0.00261799,0.999997


In [20]:
cat_pipeline.get_feature_names_out()

array(['onehotencoder__Marketing_Airline_Network_AA',
       'onehotencoder__Marketing_Airline_Network_AS',
       'onehotencoder__Marketing_Airline_Network_B6',
       'onehotencoder__Marketing_Airline_Network_DL',
       'onehotencoder__Marketing_Airline_Network_F9',
       'onehotencoder__Marketing_Airline_Network_G4',
       'onehotencoder__Marketing_Airline_Network_HA',
       'onehotencoder__Marketing_Airline_Network_NK',
       'onehotencoder__Marketing_Airline_Network_UA',
       'onehotencoder__Marketing_Airline_Network_VX',
       'onehotencoder__Marketing_Airline_Network_WN',
       'onehotencoder__Origin_ABE', 'onehotencoder__Origin_ABI',
       'onehotencoder__Origin_ABQ', 'onehotencoder__Origin_ABR',
       'onehotencoder__Origin_ACT', 'onehotencoder__Origin_ACV',
       'onehotencoder__Origin_ACY', 'onehotencoder__Origin_ADK',
       'onehotencoder__Origin_ADQ', 'onehotencoder__Origin_AEX',
       'onehotencoder__Origin_AGS', 'onehotencoder__Origin_ALB',
       'onehoten